import Libraries

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, BertModel
from transformers import pipeline
import pandas as pd
import torch
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow_datasets as tfds
import tensorflow as tf
import datasets


Load finbert Model

In [ ]:
model_name = "ProsusAI/finbert"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

print("MOdel Loaded!")


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

MOdel Loaded!


In [ ]:
df = pd.read_csv('all-data.csv', encoding='cp1252', names=['sentiment', 'news'])

accuracy = 0

for _, row in df.iterrows():
    text = row['news']
    sentiment = nlp(text)[0]['label']
    if sentiment == row['sentiment']:
        accuracy += 1

accuracy /= len(df)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 88.96%


In [ ]:
import requests
from datetime import datetime

# Define the base URL and your API key
api_key = 'e290d027a49d428fba9e805e15b0f2cc'  # Replace with your actual NewsAPI key
url = 'https://newsapi.org/v2/everything'

# Define the search parameters
company_name = 'NVIDIA'  # Example: search for Tesla news
from_date = '2024-10-15'  # Example: start date (inclusive)
to_date = '2024-11-14'  # Example: end date (inclusive)

# Set up the query parameters
params = {
    'q': company_name,  # The company or topic you're interested in
    'from': from_date,  # Start date (inclusive)
    'to': to_date,  # End date (inclusive)
    'apiKey': api_key,  # Your API key
    'language': 'en',  # Specify language (optional)
    'sortBy': 'publishedAt',  # Sort by publication date (newest first)
    'pageSize': 30,  # Limit the number of results per page (optional)
    'page': 1  # The page number of results to retrieve (optional)
}

# Make the API request
response = requests.get(url, params=params)

# Parse the response as JSON
data = response.json()

titles = []
published_dates = []

for article in data['articles']:
    titles.append(article['title'])
    published_dates.append(article['publishedAt'])

df_articles = pd.DataFrame({
    'title': titles,
    'published_date': published_dates
})


In [ ]:
import requests

# Define the API endpoint and parameters
api_key = 'csrrhm1r01qj3u0ot4v0csrrhm1r01qj3u0ot4vg'
company_symbol = 'NVDA'  # Example: Tesla
url = f'https://finnhub.io/api/v1/company-news?symbol={company_symbol}&from=2024-01-01&to=2024-12-31&token={api_key}'

# Get the data
response = requests.get(url)
data = response.json()

titles = []
published_dates = []

for article in data:  # Assuming 'data' is from your Finnhub API request
    titles.append(article['headline'])
    published_dates.append(article['datetime'])


In [ ]:
UB88LG9RO6IAUPKF

In [ ]:



# Load the pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Set the model to evaluation mode (important for inference)
model.eval()


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:
# Example list of articles (replace with your dataset)
article_titles = df_articles['title'].tolist()

# Tokenize and encode each article
inputs = tokenizer(article_titles, return_tensors='pt', padding=True, truncation=True, max_length=512)

with torch.no_grad():  # Disable gradient calculation for inference
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state

article_embeddings = embeddings.mean(dim=1).numpy()  # Average across tokens and convert to NumPy


In [ ]:
print(article_embeddings.shape)

(30, 768)


In [ ]:
df = pd.read_csv('NVIDIA.csv', names=['sentiment', 'news'])

X = df.drop(['Close', 'Date'], axis=1)
y = df['Close']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = xgb.XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

accuracy = 0


In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import requests
import transformers
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

# Load pre-trained FinBERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
model = BertModel.from_pretrained('yiyanghkust/finbert-tone')

# Function to fetch data (e.g., from NewsAPI)
def fetch_news(api_key, query, page_size=100):
    url = f"https://newsapi.org/v2/everything?q={query}&pageSize={page_size}&apiKey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return pd.DataFrame(response.json().get('articles', []))
    else:
        print("Error fetching data:", response.status_code)
        return pd.DataFrame()

# Function to preprocess news data
def preprocess_text(text):
    return tokenizer(text, padding='max_length', max_length=512, truncation=True, return_tensors='pt')

# Function to encode labels
def encode_labels(labels):
    le = LabelEncoder()
    return le.fit_transform(labels), le

# Fine-tune FinBERT and generate embeddings
def generate_embeddings(text_data):
    inputs = preprocess_text(text_data)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()  # Mean pooling for sentence embeddings

# Load datasets
def load_datasets():
    # Example loading data from Financial PhraseBank (replace with actual paths)
    df = pd.read_csv('all-data.csv', encoding='cp1252', names=['sentiment', 'news'])
    return df

# Train BERT + XGBoost
def train_model(df):
    df['sentiment_encoded'], le = encode_labels(df['sentiment'])
    embeddings = np.vstack(df['news'].apply(generate_embeddings))

    X_train, X_test, y_train, y_test = train_test_split(embeddings, df['sentiment_encoded'], test_size=0.2, random_state=42)

    xgb_model = XGBClassifier()
    xgb_model.fit(X_train, y_train)

    y_pred = xgb_model.predict(X_test)
    evaluate_model(y_test, y_pred)
    return xgb_model, le

# Evaluate model performance
def evaluate_model(y_true, y_pred):
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.2f}")
    print(f"Precision: {precision_score(y_true, y_pred, average='weighted'):.2f}")
    print(f"Recall: {recall_score(y_true, y_pred, average='weighted'):.2f}")
    print(f"AUC: {roc_auc_score(y_true, y_pred, multi_class='ovo'):.2f}")

# Correlate sentiment with stock prices
def correlate_sentiment_with_stock(df, stock_data):
    merged_df = pd.merge(df, stock_data, on='date', how='inner')
    # Example: Compute correlation
    print("Correlation with stock price movements:")
    print(merged_df.corr())

# Main function to execute the workflow
if __name__ == "__main__":
    # Step 1: Fetch financial news
    API_KEY = 'e290d027a49d428fba9e805e15b0f2cc'
    news_df = fetch_news(API_KEY, "financial market")

    # Step 2: Load and preprocess datasets
    financial_phrasebank = load_datasets()

    # Step 3: Train BERT + XGBoost
    model, label_encoder = train_model(financial_phrasebank)

    # Step 4: Fetch stock price data (example from Alpha Vantage)
    # Placeholder: Replace with actual fetching of stock price data
    stock_price_data = pd.DataFrame({'date': pd.date_range(start='2022-01-01', periods=100), 'price': np.random.rand(100)})

    # Step 5: Correlate sentiment with stock prices
    correlate_sentiment_with_stock(financial_phrasebank, stock_price_data)


Accuracy: 0.81
Precision: 0.82
Recall: 0.81


AxisError: axis 1 is out of bounds for array of dimension 1